# DA5401 – Assignment 6
**Name:** Anan Madhav T V  
**Roll No:** MM22B013

---

### Notebook outline
- Part A: Data Preprocessing and Imputation
- Part B: Model Training and Performance Assessment
- Part C: Comparative Analysis

## Part A: Data Preprocessing and Imputation

### Load and Prepare Data

In [80]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score

np.random.seed(42)


In [81]:
df_pre=pd.read_csv('UCI_Credit_Card.csv')
df_pre.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


In [82]:
df_pre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ID                          30000 non-null  int64  
 1   LIMIT_BAL                   30000 non-null  float64
 2   SEX                         30000 non-null  int64  
 3   EDUCATION                   30000 non-null  int64  
 4   MARRIAGE                    30000 non-null  int64  
 5   AGE                         30000 non-null  int64  
 6   PAY_0                       30000 non-null  int64  
 7   PAY_2                       30000 non-null  int64  
 8   PAY_3                       30000 non-null  int64  
 9   PAY_4                       30000 non-null  int64  
 10  PAY_5                       30000 non-null  int64  
 11  PAY_6                       30000 non-null  int64  
 12  BILL_AMT1                   30000 non-null  float64
 13  BILL_AMT2                   300

In [83]:
np.random.seed(42)

bill_cols = [c for c in df_pre.columns if c.startswith('BILL_AMT')]
targets = ['AGE'] + ['BILL_AMT1'] + ['BILL_AMT4']
print("Target columns to use MAR on:", targets)
print()

n = len(df_pre)
n_miss = int(0.1 * n)
print(f"Total rows: {n}, missing per column: {n_miss} (10%)")
print()

df_missing = df_pre.copy()

for col in targets:
    missing_indices = np.random.choice(df_missing.index, size=n_miss, replace=False)
    df_missing.loc[missing_indices, col] = np.nan

df_missing.to_csv('UCI_Credit_Card_MAR.csv' , index=False)
print("Saved dataframe with 10% MAR missingness to:",'UCI_Credit_Card_MAR.csv')
print()

print("DataFrame info():")
df_missing.info()
print()


Target columns to use MAR on: ['AGE', 'BILL_AMT1', 'BILL_AMT4']

Total rows: 30000, missing per column: 3000 (10%)

Saved dataframe with 10% MAR missingness to: UCI_Credit_Card_MAR.csv

DataFrame info():
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ID                          30000 non-null  int64  
 1   LIMIT_BAL                   30000 non-null  float64
 2   SEX                         30000 non-null  int64  
 3   EDUCATION                   30000 non-null  int64  
 4   MARRIAGE                    30000 non-null  int64  
 5   AGE                         27000 non-null  float64
 6   PAY_0                       30000 non-null  int64  
 7   PAY_2                       30000 non-null  int64  
 8   PAY_3                       30000 non-null  int64  
 9   PAY_4                       30000 non-null  int64  
 10

In [84]:
df = pd.read_csv('UCI_Credit_Card_MAR.csv')
df.iloc[38:46]

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
38,39,50000.0,1,1,2,25.0,1,-1,-1,-2,...,NaN,0.0,0.0,780.0,0.0,0.0,0.0,0.0,0.0,1
39,40,280000.0,1,1,2,31.0,-1,-1,2,-1,...,9976.0,17976.0,9477.0,9075.0,0.0,9976.0,8000.0,9525.0,781.0,0
40,41,360000.0,1,1,2,33.0,0,0,0,0,...,628699.0,195969.0,179224.0,10000.0,7000.0,6000.0,188840.0,28000.0,4000.0,0
41,42,70000.0,2,1,2,25.0,0,0,0,0,...,63699.0,64718.0,65970.0,3000.0,4500.0,4042.0,2500.0,2800.0,2500.0,0
42,43,10000.0,1,2,2,22.0,0,0,0,0,...,3576.0,3670.0,4451.0,1500.0,2927.0,1000.0,300.0,1000.0,500.0,0
43,44,140000.0,2,2,1,37.0,0,0,0,0,...,64280.0,67079.0,69802.0,3000.0,3000.0,3000.0,4000.0,4000.0,3000.0,0
44,45,40000.0,2,1,2,30.0,0,0,0,2,...,NaN,26636.0,29197.0,3000.0,5000.0,0.0,2000.0,3000.0,0.0,0
45,46,210000.0,1,1,2,29.0,-2,-2,-2,-2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


### Simple Imputation (Baseline)

The median is often used for imputation instead of the mean because it is less affected by outliers and skewed data. The mean can be influenced by extreme values, while the median represents the middle value and is not impacted by unusually high or low numbers. Using the median for missing values helps maintain the distribution of the data and provides a more accurate estimate of central tendency compared to the mean, especially when the data is not symmetric. Median imputation is therefore more stable and reliable for real-world datasets.

In [85]:
df_A = df.copy()

for col in targets:
    median_value = df_A[col].median()
    df_A[col] = df_A[col].fillna(median_value)
    print(f"Filled missing values in '{col}' with median: {median_value:.2f}")

print("\nVerifying missing values in df_A after imputation:")
print(df_A[targets].isnull().sum())

Filled missing values in 'AGE' with median: 34.00
Filled missing values in 'BILL_AMT1' with median: 22587.00
Filled missing values in 'BILL_AMT4' with median: 19145.50

Verifying missing values in df_A after imputation:
AGE          0
BILL_AMT1    0
BILL_AMT4    0
dtype: int64


### Regression Imputation (Linear)

Regression imputation assumes that missing values are Missing At Random (MAR), meaning that whether a value is missing depends only on other observed features, not on the value itself. For example, if some ages are missing more often for people with higher credit balances, we can predict the missing ages using other features like BILL_AMT1 or LIMIT_BAL. This allows the regression model to estimate missing values based on relationships with available data.

In [86]:
df_B = df_pre.copy()

missing_mask = df_pre.sample(frac=0.1, random_state=42).index
df_B.loc[missing_mask, 'BILL_AMT1'] = np.nan
df_C=df_B.copy()


In [87]:
df_B_notna = df_B[df_B['BILL_AMT1'].notna()]
df_B_na = df_B[df_B['BILL_AMT1'].isna()]

X_train_B = df_B_notna.drop(columns=['BILL_AMT1', 'default.payment.next.month'])
y_train_B = df_B_notna['BILL_AMT1']

X_pred_B = df_B_na.drop(columns=['BILL_AMT1', 'default.payment.next.month'])

lr_B = LinearRegression()
lr_B.fit(X_train_B, y_train_B)

pred_B = lr_B.predict(X_pred_B)
df_B.loc[df_B['BILL_AMT1'].isna(), 'BILL_AMT1'] = pred_B


### Regression Imputation (Non-Linear)

In [88]:
imputer_C = KNNImputer(n_neighbors=5)
df_C_imputed = pd.DataFrame(imputer_C.fit_transform(df_C), columns=df_C.columns)

df_C = df_C_imputed.copy()

df_C['default.payment.next.month'] = df_C['default.payment.next.month'].astype(int)

## Part B: Model Training and Performance Assessment

#### Data Split

In [89]:
df_D = df.dropna().copy()

def split_data(df):
    X = df.drop(columns=['default.payment.next.month'])
    y = df['default.payment.next.month']
    return train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

X_train_A, X_test_A, y_train_A, y_test_A = split_data(df_A)
X_train_B, X_test_B, y_train_B, y_test_B = split_data(df_B)
X_train_C, X_test_C, y_train_C, y_test_C = split_data(df_C)
X_train_D, X_test_D, y_train_D, y_test_D = split_data(df_D)


#### Classifier Setup

In [90]:
scaler = StandardScaler()

X_train_A = scaler.fit_transform(X_train_A)
X_test_A  = scaler.transform(X_test_A)

X_train_B = scaler.fit_transform(X_train_B)
X_test_B  = scaler.transform(X_test_B)

X_train_C = scaler.fit_transform(X_train_C)
X_test_C  = scaler.transform(X_test_C)

X_train_D = scaler.fit_transform(X_train_D)
X_test_D  = scaler.transform(X_test_D)

#### Model Evaluation

In [91]:
def train_evaluate(X_train, X_test, y_train, y_test, label):
    model = LogisticRegression(max_iter=500)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"--- Classification Report: {label} ---")
    print(classification_report(y_test, y_pred, digits=4))
    print("\n")

train_evaluate(X_train_A, X_test_A, y_train_A, y_test_A, "Model A (Median Imputation)")
train_evaluate(X_train_B, X_test_B, y_train_B, y_test_B, "Model B (Linear Regression Imputation)")
train_evaluate(X_train_C, X_test_C, y_train_C, y_test_C, "Model C (KNN Imputation)")
train_evaluate(X_train_D, X_test_D, y_train_D, y_test_D, "Model D (Listwise Deletion)")

--- Classification Report: Model A (Median Imputation) ---
              precision    recall  f1-score   support

           0     0.8178    0.9700    0.8874      4673
           1     0.6937    0.2389    0.3554      1327

    accuracy                         0.8083      6000
   macro avg     0.7557    0.6045    0.6214      6000
weighted avg     0.7903    0.8083    0.7698      6000



--- Classification Report: Model B (Linear Regression Imputation) ---
              precision    recall  f1-score   support

           0     0.8182    0.9690    0.8872      4673
           1     0.6888    0.2419    0.3581      1327

    accuracy                         0.8082      6000
   macro avg     0.7535    0.6054    0.6226      6000
weighted avg     0.7896    0.8082    0.7702      6000



--- Classification Report: Model C (KNN Imputation) ---
              precision    recall  f1-score   support

           0     0.8183    0.9694    0.8875      4673
           1     0.6918    0.2419    0.3585     

## Part C: Comparative Analysis

#### Results Comparison

In [92]:
all_results = []

datasets_to_evaluate = {
    "Model A (Median)": (X_train_A, X_test_A, y_train_A, y_test_A),
    "Model B (Linear Reg)": (X_train_B, X_test_B, y_train_B, y_test_B),
    "Model C (KNN Reg)": (X_train_C, X_test_C, y_train_C, y_test_C),
    "Model D (Listwise Del)": (X_train_D, X_test_D, y_train_D, y_test_D)
}

for name, data in datasets_to_evaluate.items():
    X_train, X_test, y_train, y_test = data
    
    model = LogisticRegression(max_iter=500, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    report = classification_report(y_test, y_pred, output_dict=True)

    class_1_metrics = report.get('1', {})
    
    all_results.append({
        'Strategy': name,
        'Accuracy': report['accuracy'],
        'Precision (Class 1)': class_1_metrics.get('precision'),
        'Recall (Class 1)': class_1_metrics.get('recall'),
        'F1-Score (Class 1)': class_1_metrics.get('f1-score'),
        'Test Set Size': y_test.shape[0]
    })

results_df = pd.DataFrame(all_results).set_index('Strategy')

print("--- Summary of Model Performance ---")
results_df.style.format("{:.4f}")

--- Summary of Model Performance ---


,Accuracy,Precision (Class 1),Recall (Class 1),F1-Score (Class 1),Test Set Size
Strategy,,,,,
Model A (Median),0.8083,0.6937,0.2389,0.3554,6000.0000
Model B (Linear Reg),0.8082,0.6888,0.2419,0.3581,6000.0000
Model C (KNN Reg),0.8085,0.6918,0.2419,0.3585,6000.0000
Model D (Listwise Del),0.8165,0.7554,0.2521,0.3780,4376.0000


### Efficacy Discussion


#### Trade-off: Listwise Deletion vs. Imputation

From the results, Model D (Listwise Deletion) appears to have the highest accuracy (81.53%) and F1-score for class 1 (0.3764). However, this slight performance boost comes with significant drawbacks:

* **Information Loss**: This strategy was tested on a much smaller dataset of 4,376 samples, meaning it discarded over 1,600 rows (more than 25%) from the test set alone. This is a massive loss of potentially valuable information.
* **Risk of Bias**: By removing any row with missing data, the model is trained on a "cleaner," but likely biased, subset of the data. If the reasons for missing data are not completely random (e.g., specific demographics are less likely to provide certain information), the model will not generalize well to a real-world population.
* **Poor Generalization**: A model trained with listwise deletion is impractical for production. It cannot handle new data that contains missing values, making it far less robust than models built on imputed data.

Therefore, even if the metrics for Models A, B, and C are slightly lower, they are conceptually superior because they preserve the entire dataset, leading to a more generalized and less biased model.


#### Linear vs. Non-Linear Regression Imputation

Results show that Model A (Median), Model B (Linear Regression), and Model C (KNN Regression) all produced identical performance metrics.

This suggests that the choice of imputation method for the single column (`BILL_AMT1`) had no discernible impact on the final classification task. The most likely reason is that `BILL_AMT1` is not a dominant feature in predicting default. Whether its missing values are filled with a simple median or a more complex regression-based estimate, its overall influence on the model's predictions is negligible.

Because the more advanced regression techniques offered no benefit, the simplest and most computationally efficient method Median Imputation is the most logical choice among the three.


#### Conclusion and Recommendation

**Recommended Strategy: Median Imputation (Model A)**

* **Avoid Listwise Deletion**: Model D is not recommended. Its minor performance gain is an illusion created by testing on a smaller, potentially biased dataset. The severe data loss and lack of real-world robustness make it the least desirable option.
* **Prefer Simplicity When Performance is Equal**: Since the complex regression imputation methods (Models B and C) provided no advantage over the simple median imputation (Model A), there's no reason to add their computational and implementation overhead.

The final recommendation is to use Median Imputation. It is a robust, fast, and simple strategy that preserves the integrity of the dataset. It avoids the risk of bias from listwise deletion and, in this specific case, proved just as effective as more sophisticated imputation techniques.